## Disney

* [Pixar's Foundation for Materials](https://graphics.pixar.com/library/PxrMaterialsCourse2017/)
* [RenderMan 21 Docs - Pixar Surface Materials](https://rmanwiki.pixar.com/display/REN/Pixar+Surface+Materials)
* [disney.brdf](https://github.com/wdas/brdf/blob/master/src/brdfs/disney.brdf)

# Unreal Engine 4

There are several shaders implemented in UE4: default lit, subsurface, clearcoat, hair, cloth, eye, etc. Default lit is mostly used.    
The **Default lit** material use 2 lobes, 1 diffuse and 1 specular. Contributions from 2 lobes are additive together.

There are 4 major parameters:
* Base Color
* Metallic
* Specular
* Roughness

Base Color, Metallic and Roughness is quite commonly used. Only thing that need to pay attention is Specular, it's a scaler that allow you to control the appearance of specularity for non-metal surface.

```
GBuffer.DiffuseColor = BaseColor - BaseColor * Metallic;
GBuffer.SpecularColor = lerp(0.08 * Specular.xxx, BaseColor, Metallic.xxx);

float3 StandardShading( float3 DiffuseColor, float3 SpecularColor, float3 LobeRoughness, float3 LobeEnergy, float3 L, float3 V, half3 N )
{
	float NoL = dot(N, L);
	float NoV = dot(N, V);
	float LoV = dot(L, V);
	float InvLenH = rsqrt( 2 + 2 * LoV );
	float NoH = saturate( ( NoL + NoV ) * InvLenH );
	float VoH = saturate( InvLenH + InvLenH * LoV );
	NoL = saturate(NoL);
	NoV = saturate(abs(NoV) + 1e-5);

	// Generalized microfacet specular
	float D = D_GGX( LobeRoughness[1], NoH ) * LobeEnergy[1];
	float Vis = Vis_SmithJointApprox( LobeRoughness[1], NoV, NoL );
	float3 F = F_Schlick( SpecularColor, VoH );

	float3 Diffuse = Diffuse_Lambert( DiffuseColor );

	return Diffuse * LobeEnergy[2] + (D * Vis) * F;
}
```

Layered material is implemented as parameters blend.  

# Unity